# Load Data


In [2]:
#Set up the notebook environment
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from scipy.stats import pearsonr
from scipy import signal as sig
from scipy.integrate import simps
from sklearn.model_selection import train_test_split
from scipy import signal
from scipy.signal import butter, filtfilt, welch, stft
from sklearn.metrics import mean_squared_error as mse
from scipy.io import loadmat, savemat
from scipy.integrate import simps
from scipy.integrate import simps
import warnings
import time
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

warnings.filterwarnings("ignore", message="arrays to stack must be passed as a \"sequence\" type such as list or tuple.*")

In [4]:
'''
The traninig data, contains key: 
  'train_dg': label
  'train_ecog': features
'''
# Training data
raw_training_data = loadmat('/Users/lipuchen/penn_homework/BE521/final_project/raw_training_data.mat')

# The test data 
leaderboard_data = loadmat('/Users/lipuchen/penn_homework/BE521/final_project/leaderboard_data.mat')

# Filter Data

In [5]:
def filter_data(raw_eeg, fs=1000):
    """
    Write a filter function to clean underlying data.
    Filter type and parameters are up to you. Points will be awarded for reasonable filter type, parameters and application.
    Please note there are many acceptable answers, but make sure you aren't throwing out crucial data or adversly
    distorting the underlying data!

    Input: 
        raw_eeg (samples x channels): the raw signal
        fs: the sampling rate (1000 for this dataset)
    Output: 
        clean_data (samples x channels): the filtered signal
    """
    b = sig.firwin(numtaps=101, cutoff=[0.15, 200], pass_zero='bandpass', fs=fs)
    filtered_eeg = filtfilt(b, 1, x = raw_eeg, axis=0)    
    return filtered_eeg

# Feature extraction


In [6]:
def NumWins(signal, fs, window_len, win_overlap):
  length = (signal.shape[0] - window_len * fs)
  _olap = win_overlap * fs
  return int(1 +  length/ (_olap))

In [7]:
def average_time(x):
    return np.mean(x, axis=0)
def locomotor_potential(x):
    """
        Calculates the locomotor potential feature of the input signal x. 
        This feature represents the potential for movement-related activity.

        Input: 
        x (window_samples x channels): the window of the filtered ecog signal 

        Output:
        lp (1 x channels): the locomotor potential feature calculated for each channel
    """

    # Define the frequency bands to use for the calculation
    freq_bands = [(8, 12), (18, 24), (75, 115), (125, 159), (159, 175)]

    # Calculate the power in each frequency band for each channel
    freq_power = []
    for band in freq_bands:
        fmin, fmax = band
        power = bandpower(x, fs=1000, fmin=fmin, fmax=fmax)
        freq_power.append(power)

    # Calculate the locomotor potential feature for each channel
    lp = np.mean(freq_power, axis=0)

    return lp


def bandpower(x, fs, fmin, fmax):
    fs = 1000
    freqs, psd = sig.welch(x, fs, axis=0, nperseg=x.shape[0])
    idx_delta = np.logical_and(freqs >= fmin, freqs <= fmax)
    _freq = freqs[1] - freqs[0]
    delta_power = simps(psd[idx_delta], dx=_freq, axis=0)    
    return delta_power
    

def spectral_entropy(x, sf, method='fft', nperseg=None, normalize=False, axis=-1):
    """Calculate the spectral entropy of a signal."""
    from scipy.signal import welch

    x = np.atleast_1d(x)
    if len(x) < 2:
        return 0.0

    # Compute the power spectrum
    if method == 'fft':
        f, Pxx = welch(x, sf, nperseg=nperseg, axis=axis)
    else:
        raise ValueError("Method must be 'fft'.")

    # Normalize the power spectrum
    if normalize:
        Pxx /= Pxx.sum()

    # Calculate the spectral entropy
    spectral_entropy = -np.sum(np.multiply(Pxx, np.log2(Pxx)), axis=axis)

    return spectral_entropy


def get_features(filtered_window, fs=1000):
    """
        Write a function that calculates features for a given filtered window. 
        Feel free to use features you have seen before in this class, features that
        have been used in the literature, or design your own!

        Input: 
        filtered_window (window_samples x channels): the window of the filtered ecog signal 
        fs: sampling rate
        Output:
        features (channels x num_features): the features calculated on each channel for the window
    """
    feat_TimeAvg = average_time(filtered_window)
    feat_LMP = locomotor_potential(filtered_window)
    return np.hstack([                      
                      bandpower(filtered_window, 1000, 5, 15),
                      bandpower(filtered_window, 1000, 20, 25),
                      bandpower(filtered_window, 1000, 75, 115),
                      bandpower(filtered_window, 1000, 125, 160),
                      bandpower(filtered_window, 1000, 160, 175),
                      feat_TimeAvg])


In [8]:
def get_windowed_feats(raw_ecog, fs, window_length, window_overlap):
    """
        Write a function which processes data through the steps of filtering and
        feature calculation and returns features. Points will be awarded for completing
        each step appropriately (note that if one of the functions you call within this script
        returns a bad output, you won't be double penalized). Note that you will need
        to run the filter_data and get_features functions within this function. 

        Inputs:
        raw_eeg (samples x channels): the raw signal
        fs: the sampling rate (1000 for this dataset)
        window_length: the window's length
        window_overlap: the window's overlap
        Output: 
        all_feats (num_windows x (channels x features)): the features for each channel for each time window
            note that this is a 2D array. 
    """
    raw_ecog = filter_data(raw_ecog, fs)    
    window_disp = window_length - window_overlap
    
    all_feats = np.vstack([get_features(raw_ecog[int(i * window_disp * fs):int(i * window_disp * fs + window_length * fs), :], fs) for i in range(NumWins(raw_ecog, fs, window_length, window_overlap))])
    
    return all_feats

In [9]:
def create_R_matrix(features, N_wind):
  """ 
  Write a function to calculate the R matrix

  Input:
    features (samples (number of windows in the signal) x channels x features): 
      the features you calculated using get_windowed_feats
    N_wind: number of windows to use in the R matrix

  Output:
    R (samples x (N_wind*channels*features))
  """

  features = np.append(features,features[0:N_wind-1, :], axis=0)
  n = features.shape[0]
  _features = np.append(features[:N_wind-1, :].copy(), features, axis=0)

  R = np.stack((_features[i: i+N_wind, :]).reshape(-1) for i in range(n))
  _ones = np.ones((n, 1))
  R = np.hstack((_ones, R))

  return R

# ML Training

Process the Data and Choose the ratio for the data

In [17]:
# setup training data and test data
# Changed to True if we're doing for validation purpose
validate = False
if not validate:
  training_ratio = 1
else:
  training_ratio = 0.75


train_data = {'train_dg':[], 'train_ecog':[]}
validate_data = {'train_dg':[], 'train_ecog':[]}
fs = 1000

for key in list(raw_training_data)[-2:]:
    for i in range(3):
        r,c = raw_training_data[key][i][0].shape
        train_data[key].append(raw_training_data[key][i][0][:int(r*(training_ratio)), :])
        validate_data[key].append(raw_training_data[key][i][0][int(r*(training_ratio)):, :])

print("Number of samples in training set:", np.size(train_data['train_ecog'][0][:,0]))
print("Number of samples in validate set:", np.size(validate_data['train_ecog'][0][:,0]))

train_dg
train_dg
train_dg
train_ecog
train_ecog
train_ecog
Number of samples in training set: 300000
Number of samples in validate set: 0


In [22]:
train_data['train_dg'][0] = np.delete(train_data['train_dg'][0], 3, 1)
train_data['train_dg'][1] = np.delete(train_data['train_dg'][1], 3, 1)
train_data['train_dg'][2] = np.delete(train_data['train_dg'][2], 3, 1)

In [24]:
train_data['train_dg'][2].shape

(300000, 4)

In [25]:
N_wind = 5
winLen = 100 / 1e3
winOverlap = 50 / 1e3
winDisp = winLen - winOverlap

In [26]:

predicted_dg = {'predicted_dg': []}

if not validate:
    for j in range(3):
        print('Subject', j)
        # create train linear filter
        _train_data = train_data['train_ecog'][j]
        feature_train = get_windowed_feats(_train_data, 1000, winLen, winOverlap)
        R_train = create_R_matrix(feature_train, N_wind)

        # the training output label
        _Y_train = train_data['train_dg'][j]
        _Y_train = sig.resample(_Y_train, R_train.shape[0], axis=0)

        # Calculate the linear filter
        Y_train = np.vstack([sig.resample(_Y_train[:, i], R_train.shape[0], axis=0) for i in range(5)])
        f_train = np.linalg.pinv(R_train.T @ R_train) @ (R_train.T @ Y_train.T)

        # Apply regularization
        alpha = 5*10**(-4)*0.001
        if j == 2:
            alpha = 10**(-9)
        mean_arr = np.mean(f_train, axis=0)
        f_train += alpha * mean_arr

        # use linear regression to get initial prediction
        test_data = leaderboard_data['leaderboard_ecog'][j][0]
        feature_test = get_windowed_feats(test_data, 1000, winLen, winOverlap)
        R_test = create_R_matrix(feature_test, N_wind)
        prediction_LR_leaderboard = R_test @ f_train

        # train XGBoost model
        dtrain = xgb.DMatrix(R_train, label=Y_train)

        # set up XGBoost hyperparameters
        param = {'max_depth': 5, 'eta': 0.3, 'objective': 'reg:squarederror'}
        num_round = 20

        # train XGBoost model
        bst = xgb.train(param, dtrain, num_round)

        # use XGBoost model to predict on test data
        dtest = xgb.DMatrix(R_test)
        prediction_XGB_leaderboard = bst.predict(dtest).reshape((-1, 5))

        # combine linear regression and XGBoost predictions
        prediction_final_leaderboard = 0.5 * prediction_LR_leaderboard + 0.5 * prediction_XGB_leaderboard
        predicted_dg['predicted_dg'].append([prediction_final_leaderboard])

    print('Finished model training')

else:
  performance = []
  
  for j in range(3):
      # print('Subject', j)

      # create train linear filter
      _train_data = train_data['train_ecog'][j]
      feature_train = get_windowed_feats(_train_data, 1000, winLen, winOverlap)
      R_train = create_R_matrix(feature_train, N_wind)

      # the training output label
      _Y_train = train_data['train_dg'][j]
      _Y_train = sig.resample(_Y_train, R_train.shape[0], axis=0)

      # Calculate the linear filter
      Y_train = np.vstack([sig.resample(_Y_train[:, i], R_train.shape[0], axis=0) for i in range(5)])
      
      f_train = np.linalg.pinv(R_train.T @ R_train) @ (R_train.T @ Y_train.T)
      # Apply regularization

      alpha = 5*10**(-4)*0.001
      if j == 2:
          alpha = 10**(-9)
      mean_arr = np.mean(f_train, axis=0)
      f_train += alpha * mean_arr

      # predict on validation data
      _validate_data = validate_data['train_ecog'][j]
      feature_validate = get_windowed_feats(_validate_data, 1000, winLen, winOverlap)
      R_validate = create_R_matrix(feature_validate, N_wind)

      # the validation output label
      _Y_validate = validate_data['train_dg'][j]
      _Y_validate = sig.resample(_Y_validate, R_validate.shape[0], axis=0)

      # predict the output
      Y_validate = np.vstack([sig.resample(_Y_validate[:, i], R_validate.shape[0], axis=0) for i in range(5)])
      prediction_LR_validate = R_validate @ f_train

      # calculate performance
      perf, _ = pearsonr(prediction_LR_validate.flatten(), Y_validate.T.flatten())
      performance.append(perf)

      if j != 2:
        print('Validation performance for subject', j, ':', perf, tune)
  print('Overall validation performance:', np.mean(performance))


# Save the predictions
savemat('_predicted_dg.mat', predicted_dg)

In [17]:
savemat('_predicted_dg.mat', predicted_dg)

```
alpha = 5*10**(-4)*0.001
    if j == 2:
      alpha = 10**(-9)
```

0.4260739147939815

In [ ]:
predicted_dg = {'predicted_dg': []}

if not validate:
  for j in range(3):

    print('Subject', j)
    # create train linear filter
    _train_data = train_data['train_ecog'][j]
    feature_train = get_windowed_feats(_train_data, 1000, winLen, winOverlap)
    R_train = create_R_matrix(feature_train, N_wind)
    print(R_train.shape)

    # the training output label
    _Y_train = train_data['train_dg'][j]
    _Y_train = sig.resample(_Y_train, R_train.shape[0], axis=0)

    # Train XGBoost model on R_train and Y_train
    model = xgb.XGBRegressor(n_estimators=100, max_depth=5)
    model.fit(R_train, _Y_train)

    test_data = leaderboard_data['leaderboard_ecog'][j][0]

    # Compute the R matrix for the test data(leaderboard data)  
    feature_test = get_windowed_feats(test_data, 1000, winLen, winOverlap)
    R_test = create_R_matrix(feature_test, N_wind)
    
    # Make the predictions using the trained model
    prediction_xgb_leaderboard = model.predict(R_test)
    
    predicted_dg['predicted_dg'].append([prediction_xgb_leaderboard])
  
  print('Finished Model Training')
else:
  performance = []
  for j in range(3):
      print('Subject', j)

      # create train linear filter
      _train_data = train_data['train_ecog'][j]
      feature_train = get_windowed_feats(_train_data, 1000, winLen, winOverlap)
      R_train = create_R_matrix(feature_train, N_wind)

      # the training output label
      _Y_train = train_data['train_dg'][j]
      _Y_train = sig.resample(_Y_train, R_train.shape[0], axis=0)

      # Train XGBoost model on R_train and Y_train
      model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, max_depth=5)
      model.fit(R_train, Y_train)

      # predict on validation data
      _validate_data = validate_data['train_ecog'][j]
      feature_validate = get_windowed_feats(_validate_data, 1000, winLen, winOverlap)
      R_validate = create_R_matrix(feature_validate, N_wind)

      # the validation output label
      _Y_validate = validate_data['train_dg'][j]
      _Y_validate = sig.resample(_Y_validate, R_validate.shape[0], axis=0)

      # predict the output using the trained model
      prediction_xgb_validate = model.predict(R_validate)

      # calculate performance
      perf, _ = pearsonr(prediction_xgb_validate.flatten(), Y_validate.T.flatten())
      performance.append(perf)
      print('Validation performance for subject', j, ':', perf)

  print('Overall validation performance:', performance)

# Save the predictions
savemat('_predicted_dg.mat', predicted_dg)


In [14]:
if not validate:
  # Leveraging this block to upsample to 147000
  from scipy.interpolate import interp1d

  # load the predicted_dg data

  predicted_dg = loadmat('_predicted_dg.mat')['predicted_dg']


  upsampled_dg = []
  for i in range(3):
      subject_dg = predicted_dg[i, 0, :, :]
      print(f'subject {i}: shape of dg: {subject_dg.shape}')
      upsampled_subject_dg = np.zeros((147500, 5))
      for j in range(5):
          x = np.arange(subject_dg[:, j].shape[0])
          y = subject_dg[:, j]
          if i == 2:
            f = interp1d(x, y, kind='cubic')  
          else:
            f = interp1d(x, y, kind='linear')
          x_new = np.linspace(0, subject_dg[:, j].shape[0]-1, num=147500)
          upsampled_subject_dg[:, j] = f(x_new)
      upsampled_dg.append(upsampled_subject_dg)
  
  upsampled_predicted_dg = {'predicted_dg': np.array(upsampled_dg).reshape((3, 1, 147500, 5))}
  savemat('predicted_dg.mat', upsampled_predicted_dg)
  

subject 0: shape of dg: (2953, 5)
subject 1: shape of dg: (2953, 5)
subject 2: shape of dg: (2953, 5)


In [ ]:
# if not validate:

#   from scipy.ndimage import gaussian_filter1d
#   # Assuming upsampled data is stored in 'upsampled_dg' variable
#   smoothed_dg = []
#   for i in range(3):
#       subject_dg = upsampled_dg[i]
#       smoothed_subject_dg = np.zeros((subject_dg.shape[0], 5))
#       for j in range(5):
#           smoothed_subject_dg[:, j] = gaussian_filter1d(subject_dg[:, j], sigma=7) # present: 7 
#       smoothed_dg.append(smoothed_subject_dg)

#   # Replace upsampled data with smoothed data
#   upsampled_dg = smoothed_dg
#   # save the upsampled predicted_dg data
#   upsampled_predicted_dg = {'predicted_dg': np.array(upsampled_dg).reshape((3, 1, 147500, 5))}
#   savemat('predicted_dg.mat', upsampled_predicted_dg)


In [15]:
# Print to check if this upsaple is good for shape: (3, 1, 147500, 5)
if not validate:
  up_x = loadmat('predicted_dg.mat')
  print(up_x['predicted_dg'].shape)

(3, 1, 147500, 5)


In [ ]:
# import matplotlib.pyplot as plt

# # Assuming upsampled data is stored in 'upsampled_dg' variable
# fig, axs = plt.subplots(2, 2, figsize=(10, 10))
# sigmas = [1, 32]

# for i in range(3):
#     subject_dg = upsampled_dg[i]
#     axs[i//2, i%2].plot(subject_dg[:, 0], label='Original')
#     for sigma in sigmas:
#         smoothed_subject_dg = gaussian_filter1d(subject_dg[:, 0], sigma=sigma)
#         axs[i//2, i%2].plot(smoothed_subject_dg, label=f'Sigma = {sigma}')
#     axs[i//2, i%2].set_title(f'Subject {i}')
#     axs[i//2, i%2].legend()

# plt.show()
